In [10]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [22]:
subject = 'Goiás - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [23]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_GO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Goiás - Produção de Cimento (t)', axis=1)
data

,Ano,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - Consumo de Cimento (t)
0,2008,0.750753,0.786865,0.740091,9.979485e+07,5.727076e+06,14.203138,8.592012e+07,0.726402,7.941146,...,2.868108,11.760983,25.567101,931.224932,1376.621797,23.533988,16.021792,67.029130,1.258587,2310.616000
1,2009,0.751479,0.787352,0.738646,1.005138e+08,5.757645e+06,14.197250,8.596210e+07,0.729972,7.969157,...,2.305885,9.472585,26.358065,931.509364,1404.486199,23.539793,15.895700,66.852293,1.302609,2335.717000
2,2010,0.752088,0.787273,0.737500,1.030813e+08,5.806966e+06,14.241313,8.663480e+07,0.733384,7.932921,...,1.958246,9.332485,26.324590,931.774922,1422.397943,23.407790,15.594180,66.528157,1.346857,2657.527000
3,2011,0.752581,0.786636,0.736643,1.054535e+08,5.798734e+06,14.278825,8.718791e+07,0.736630,7.921619,...,2.079088,10.985689,26.891246,932.026640,1424.732773,23.379311,15.713953,67.017525,1.343835,2901.707000
4,2012,0.752723,0.785429,0.735750,1.075784e+08,5.775231e+06,14.289292,8.762067e+07,0.739409,7.786290,...,2.183282,8.159014,28.111519,932.153867,1426.391027,23.445549,15.826075,67.022758,1.300550,3061.533000
5,2013,0.753450,0.783632,0.736229,1.092519e+08,5.758466e+06,14.261467,8.779968e+07,0.742860,8.389968,...,2.403445,7.913561,27.073883,932.317691,1424.468340,23.563897,15.775697,66.406360,1.252025,3181.778000
6,2014,0.753571,0.781434,0.735915,1.105942e+08,5.711944e+06,14.238295,8.787916e+07,0.745598,9.015756,...,2.748397,10.350442,25.841819,932.468684,1423.836782,23.878739,15.683323,65.088818,1.241766,2861.897429
7,2015,0.753286,0.778804,0.735658,1.115113e+08,5.628820e+06,14.206629,8.784849e+07,0.747481,9.398111,...,2.501529,12.454398,25.709892,932.508663,1430.027526,24.014961,15.540525,64.054609,1.245195,2517.290743
8,2016,0.756276,0.784027,0.728777,1.120977e+08,5.573901e+06,14.184179,8.781949e+07,0.758668,9.378450,...,2.073464,13.111908,26.601698,932.486431,1433.688880,23.900472,15.251199,63.441573,1.236011,2193.545829
9,2017,0.755573,0.780939,0.729212,1.123228e+08,5.555871e+06,14.177721,8.783412e+07,0.759092,8.980786,...,2.177386,9.479812,26.603489,932.515580,1426.852240,23.876158,15.213263,63.423776,1.188108,2270.346000


In [24]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.750753,0.786865,0.740091,9.979485e+07,5.727076e+06,14.203138,8.592012e+07,0.726402,7.941146,5.483335e+08,...,-7723.574973,2.868108,11.760983,25.567101,931.224932,1376.621797,23.533988,16.021792,67.029130,1.258587
1,0.751479,0.787352,0.738646,1.005138e+08,5.757645e+06,14.197250,8.596210e+07,0.729972,7.969157,5.302764e+08,...,-8961.781709,2.305885,9.472585,26.358065,931.509364,1404.486199,23.539793,15.895700,66.852293,1.302609
2,0.752088,0.787273,0.737500,1.030813e+08,5.806966e+06,14.241313,8.663480e+07,0.733384,7.932921,5.046425e+08,...,-9895.682385,1.958246,9.332485,26.324590,931.774922,1422.397943,23.407790,15.594180,66.528157,1.346857
3,0.752581,0.786636,0.736643,1.054535e+08,5.798734e+06,14.278825,8.718791e+07,0.736630,7.921619,4.854622e+08,...,-9989.170714,2.079088,10.985689,26.891246,932.026640,1424.732773,23.379311,15.713953,67.017525,1.343835
4,0.752723,0.785429,0.735750,1.075784e+08,5.775231e+06,14.289292,8.762067e+07,0.739409,7.786290,4.867661e+08,...,-9429.326328,2.183282,8.159014,28.111519,932.153867,1426.391027,23.445549,15.826075,67.022758,1.300550
5,0.753450,0.783632,0.736229,1.092519e+08,5.758466e+06,14.261467,8.779968e+07,0.742860,8.389968,5.225848e+08,...,-7157.441850,2.403445,7.913561,27.073883,932.317691,1424.468340,23.563897,15.775697,66.406360,1.252025
6,0.753571,0.781434,0.735915,1.105942e+08,5.711944e+06,14.238295,8.787916e+07,0.745598,9.015756,5.676413e+08,...,-3453.461327,2.748397,10.350442,25.841819,932.468684,1423.836782,23.878739,15.683323,65.088818,1.241766
7,0.753286,0.778804,0.735658,1.115113e+08,5.628820e+06,14.206629,8.784849e+07,0.747481,9.398111,5.179392e+08,...,-1442.919387,2.501529,12.454398,25.709892,932.508663,1430.027526,24.014961,15.540525,64.054609,1.245195
8,0.756276,0.784027,0.728777,1.120977e+08,5.573901e+06,14.184179,8.781949e+07,0.758668,9.378450,4.524144e+08,...,-3128.377228,2.073464,13.111908,26.601698,932.486431,1433.688880,23.900472,15.251199,63.441573,1.236011
9,0.755573,0.780939,0.729212,1.123228e+08,5.555871e+06,14.177721,8.783412e+07,0.759092,8.980786,4.213689e+08,...,-5386.337108,2.177386,9.479812,26.603489,932.515580,1426.852240,23.876158,15.213263,63.423776,1.188108


In [25]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2335.717000
1     2657.527000
2     2901.707000
3     3061.533000
4     3181.778000
5     2861.897429
6     2517.290743
7     2193.545829
8     2270.346000
9     2282.903000
10    2367.716000
11    2438.585000
12    2782.135000
13    2807.973000
14            NaN
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [26]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.667342,0.949720,1.410033,-1.833449,0.433923,-0.325155,-1.986396,-1.569745,-1.027093,1.204752,...,-0.569731,2.026352,0.984445,-1.331393,-2.215071,-2.920210,-0.729782,1.542463,1.077765,0.191401
1,-1.194211,1.061657,1.060251,-1.672467,0.753723,-0.464589,-1.926962,-1.269707,-0.979720,0.820657,...,-0.838980,0.085558,-0.051710,-0.314120,-1.543139,-1.011487,-0.705275,1.108832,0.970139,0.769359
2,-0.797383,1.043475,0.782547,-1.097605,1.269707,0.578899,-0.974517,-0.982875,-1.041004,0.275392,...,-1.042057,-1.114490,-0.115145,-0.357173,-0.915796,0.215476,-1.262514,0.071898,0.772865,1.350269
3,-0.475496,0.897221,0.575051,-0.566452,1.183590,1.467272,-0.191390,-0.710011,-1.060119,-0.132596,...,-1.062386,-0.697344,0.633403,0.371613,-0.321146,0.375413,-1.382735,0.483799,1.070702,1.310592
4,-0.382925,0.620129,0.358866,-0.090684,0.937709,1.715155,0.421338,-0.476466,-1.288995,-0.104860,...,-0.940648,-0.337667,-0.646476,1.941026,-0.020591,0.489004,-1.103118,0.869388,1.073887,0.742316
5,0.090857,0.207420,0.474906,0.284004,0.762311,1.056209,0.674789,-0.186418,-0.268017,0.657046,...,-0.446625,0.422337,-0.757614,0.606505,0.366422,0.357299,-0.603521,0.696138,0.698737,0.105254
6,0.169632,-0.297081,0.398809,0.584563,0.275613,0.507437,0.787318,0.043747,0.790354,1.615453,...,0.358807,1.613108,0.345772,-0.978073,0.723121,0.314037,0.725548,0.378463,-0.103139,-0.029436
7,-0.016584,-0.901022,0.336552,0.789891,-0.594012,-0.242472,0.743896,0.202034,1.437016,0.558230,...,0.796000,0.760923,1.298413,-1.147746,0.817566,0.738105,1.300597,-0.112621,-0.732575,0.015582
8,1.932066,0.298219,-1.329669,0.921200,-1.168560,-0.774142,0.702846,1.142334,1.403764,-0.835563,...,0.429496,-0.716758,1.596124,-0.000780,0.765046,0.988910,0.817294,-1.107619,-1.105679,-0.104987
9,1.474402,-0.410869,-1.224492,0.971602,-1.357181,-0.927081,0.723547,1.178003,0.731211,-1.495939,...,-0.061498,-0.358019,-0.048438,0.001524,0.833907,0.520597,0.714653,-1.238081,-1.116510,-0.733882


In [27]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     2335.717000
1     2657.527000
2     2901.707000
3     3061.533000
4     3181.778000
5     2861.897429
6     2517.290743
7     2193.545829
8     2270.346000
9     2282.903000
10    2367.716000
11    2438.585000
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [28]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.158228,-2.357097,-1.338521,0.495067,-0.996697,-1.327004,0.145995,1.141532,0.318701,1.108405,...,0.964765,-1.482779,-2.260447,0.946592,0.252129,0.101107,0.76294,-1.953922,-1.523095,-2.072592


In [29]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2782.135
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [30]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [31]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [33]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3856663030, 414631537, 3230828399, 2833761860, 483974093, 1908629510, 2674524835, 1536333784, 383479515, 3384445213, 1256207522, 2176606654, 296680092, 3916941154, 2374794142, 3092222602, 180954045, 3243198016, 486759192, 2629854296, 3964717981, 998593948, 2149071589, 2630302263, 3664336313, 491164003, 2374876667, 1157270496, 757802177, 3113956456, 1935189756, 1443479671, 675186274, 1866899677, 2360488421, 3481425455, 33390442, 1186032148, 292235785, 4248802394, 3242067857, 3898127038, 389173076, 1303519399, 2339651042, 2755285288, 318103811, 1593976606, 3366222784, 2045412005, 2325371625, 1225655900, 2119411450, 2340914444, 3697534410, 3844919710, 2777889240, 3688825067, 3435431602, 3504665949, 2842367219, 2812101355, 2415666838, 1367950629, 3133609904, 2181484140, 3871645982, 4256688471, 1355105077, 2931049997, 1848529795, 170248148, 1869026764, 3936306580, 1085953322, 3932782262, 2663205908, 4097585407, 4106751335, 2047632928, 2535727658, 3333968234, 44230141, 1795974733, 193838226

2023-09-14 21:06:53.396137: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 21:06:53.424339: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 21:06:53.455557: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 21:06:53.945049: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 21:06:53.965237: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 21:06:53.994962: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 1412.79443359375


Step: 8 ___________________________________________
val_loss: 3699.704345703125


Step: 9 ___________________________________________
val_loss: 4881.79931640625


Step: 10 ___________________________________________
val_loss: 3893.1689453125


Step: 11 ___________________________________________
val_loss: 11289.2177734375


Step: 12 ___________________________________________
val_loss: 4727.2001953125


Step: 13 ___________________________________________
val_loss: 0.02691650390625
winner_seed: 3916941154


Step: 14 ___________________________________________
val_loss: 4490.7666015625


Step: 15 ___________________________________________
val_loss: 3861.244873046875


Step: 16 ___________________________________________
val_loss: 7491.8828125


Step: 17 ___________________________________________
val_loss: 1413.1982421875


Step: 18 ___________________________________________
val_loss: 8129.2060546875


Step: 19 ___________________________________________
v

In [34]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 486ms/step - loss: 6807190.5000 - val_loss: 5673447.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 6627679.5000 - val_loss: 7466040.0000
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 6732157.5000 - val_loss: 8236121.0000
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 6112286.0000 - val_loss: 8157066.5000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 5843068.5000 - val_loss: 9488151.0000
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 6067630.5000 - val_loss: 8002814.0000
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 5565490.0000 - val_loss: 9188530.0000
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 5558675.5000 - val_loss: 8549328.0000
Epoch 9/10000
1/1 [==============================] - 0s 15ms/step - loss: 5449502.5000 - val_loss: 9959

1/1 [==============================] - 0s 14ms/step - loss: 1407547.5000 - val_loss: 1072027.1250
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 1608334.1250 - val_loss: 846530.7500
Epoch 75/10000
1/1 [==============================] - 0s 15ms/step - loss: 1402158.0000 - val_loss: 1165830.7500
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 1807760.0000 - val_loss: 1680154.2500
Epoch 77/10000
1/1 [==============================] - 0s 15ms/step - loss: 1403128.8750 - val_loss: 1322619.0000
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 2179217.2500 - val_loss: 1264749.0000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 1718340.6250 - val_loss: 867079.5000
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 1363781.6250 - val_loss: 1287933.0000
Epoch 81/10000
1/1 [==============================] - 0s 15ms/step - loss: 1290281.1250 - val_loss: 1152965.2500


1/1 [==============================] - 0s 15ms/step - loss: 497081.9062 - val_loss: 387361.2812
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 459998.8750 - val_loss: 396381.8438
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 432747.5000 - val_loss: 571640.5000
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 432413.2812 - val_loss: 396802.1875
Epoch 150/10000
1/1 [==============================] - 0s 15ms/step - loss: 516438.7812 - val_loss: 370451.1250
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 481303.6250 - val_loss: 411530.7188
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 429040.2500 - val_loss: 326613.5000
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 521788.4688 - val_loss: 334361.0625
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 379159.8438 - val_loss: 763382.5625
Epoch 15

1/1 [==============================] - 0s 14ms/step - loss: 215374.0469 - val_loss: 120265.8750
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 210519.1875 - val_loss: 101175.3984
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 177492.7031 - val_loss: 184743.4062
Epoch 222/10000
1/1 [==============================] - 0s 15ms/step - loss: 203150.2500 - val_loss: 110498.4609
Epoch 223/10000
1/1 [==============================] - 0s 15ms/step - loss: 199438.9531 - val_loss: 113703.8047
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 187182.2969 - val_loss: 178580.6250
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 169881.7344 - val_loss: 162486.6406
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 202935.3906 - val_loss: 116332.1250
Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 199898.5000 - val_loss: 111907.8906
Epoch 22

1/1 [==============================] - 0s 14ms/step - loss: 106238.0391 - val_loss: 65043.4297
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 97066.8750 - val_loss: 68086.7188
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 104880.6875 - val_loss: 50519.2578
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 106700.9453 - val_loss: 62347.2617
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 88376.0859 - val_loss: 97983.9766
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 125880.7812 - val_loss: 64959.7734
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 93331.8359 - val_loss: 81972.6094
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 103132.4375 - val_loss: 64490.5781
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 96988.4141 - val_loss: 84037.0000
Epoch 301/10000
1/1 [

...

1/1 [==============================] - 0s 14ms/step - loss: 41963.4648 - val_loss: 1254.7266
Epoch 1483/10000
1/1 [==============================] - 0s 14ms/step - loss: 42096.9531 - val_loss: 1737.8884
Epoch 1484/10000
1/1 [==============================] - 0s 14ms/step - loss: 38432.7930 - val_loss: 8376.8525
Epoch 1485/10000
1/1 [==============================] - 0s 14ms/step - loss: 42797.2617 - val_loss: 3646.9980
Epoch 1486/10000
1/1 [==============================] - 0s 15ms/step - loss: 43855.4102 - val_loss: 8308.0381
Epoch 1487/10000
1/1 [==============================] - 0s 14ms/step - loss: 44478.9180 - val_loss: 14806.4219
Epoch 1488/10000
1/1 [==============================] - 0s 14ms/step - loss: 46434.5938 - val_loss: 2713.5044
Epoch 1489/10000
1/1 [==============================] - 0s 14ms/step - loss: 44362.0508 - val_loss: 7786.4683
Epoch 1490/10000
1/1 [==============================] - 0s 14ms/step - loss: 43347.8789 - val_loss: 8832.4658
Epoch 1491/10000
1/1 [====

1/1 [==============================] - 0s 14ms/step - loss: 41942.1406 - val_loss: 3330.9187
Epoch 1557/10000
1/1 [==============================] - 0s 15ms/step - loss: 42093.1406 - val_loss: 5697.4854
Epoch 1558/10000
1/1 [==============================] - 0s 15ms/step - loss: 41795.7617 - val_loss: 5129.9658
Epoch 1559/10000
1/1 [==============================] - 0s 14ms/step - loss: 39066.7305 - val_loss: 3963.8337
Epoch 1560/10000
1/1 [==============================] - 0s 14ms/step - loss: 42568.8359 - val_loss: 4233.7642
Epoch 1561/10000
1/1 [==============================] - 0s 14ms/step - loss: 41961.2109 - val_loss: 3889.5747
Epoch 1562/10000
1/1 [==============================] - 0s 14ms/step - loss: 41212.7461 - val_loss: 5623.7915
Epoch 1563/10000
1/1 [==============================] - 0s 14ms/step - loss: 41537.5664 - val_loss: 8531.0205
Epoch 1564/10000
1/1 [==============================] - 0s 14ms/step - loss: 43866.5273 - val_loss: 1763.3234
Epoch 1565/10000
1/1 [=====

1/1 [==============================] - 0s 15ms/step - loss: 41324.0117 - val_loss: 4279.6704
Epoch 1631/10000
1/1 [==============================] - 0s 14ms/step - loss: 40205.7500 - val_loss: 4651.7329
Epoch 1632/10000
1/1 [==============================] - 0s 14ms/step - loss: 39069.9648 - val_loss: 7473.9316
Epoch 1633/10000
1/1 [==============================] - 0s 14ms/step - loss: 43389.5508 - val_loss: 3754.7034
Epoch 1634/10000
1/1 [==============================] - 0s 14ms/step - loss: 41903.8555 - val_loss: 2561.0864
Epoch 1635/10000
1/1 [==============================] - 0s 14ms/step - loss: 38962.6758 - val_loss: 3766.1714
Epoch 1636/10000
1/1 [==============================] - 0s 14ms/step - loss: 41748.1992 - val_loss: 3376.3022
Epoch 1637/10000
1/1 [==============================] - 0s 15ms/step - loss: 40847.4648 - val_loss: 5799.6631
Epoch 1638/10000
1/1 [==============================] - 0s 15ms/step - loss: 43439.7617 - val_loss: 2719.6887
Epoch 1639/10000
1/1 [=====

1/1 [==============================] - 0s 15ms/step - loss: 41583.5820 - val_loss: 6128.2983
Epoch 1705/10000
1/1 [==============================] - 0s 14ms/step - loss: 44953.3438 - val_loss: 3557.6741
Epoch 1706/10000
1/1 [==============================] - 0s 14ms/step - loss: 41898.7930 - val_loss: 5528.4453
Epoch 1707/10000
1/1 [==============================] - 0s 14ms/step - loss: 40905.5352 - val_loss: 8165.0815
Epoch 1708/10000
1/1 [==============================] - 0s 14ms/step - loss: 40947.9297 - val_loss: 2415.8088
Epoch 1709/10000
1/1 [==============================] - 0s 14ms/step - loss: 41724.0117 - val_loss: 3918.2527
Epoch 1710/10000
1/1 [==============================] - 0s 14ms/step - loss: 42092.0352 - val_loss: 2160.3433
Epoch 1711/10000
1/1 [==============================] - 0s 14ms/step - loss: 41945.5312 - val_loss: 4406.5806
Epoch 1712/10000
1/1 [==============================] - 0s 14ms/step - loss: 40487.6523 - val_loss: 5445.8706
Epoch 1713/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 41376.8828 - val_loss: 4355.6777
Epoch 1779/10000
1/1 [==============================] - 0s 14ms/step - loss: 42371.5273 - val_loss: 1538.7998
Epoch 1780/10000
1/1 [==============================] - 0s 14ms/step - loss: 41074.4141 - val_loss: 4910.0195
Epoch 1781/10000
1/1 [==============================] - 0s 13ms/step - loss: 40887.7305 - val_loss: 1748.7344
Epoch 1782/10000
1/1 [==============================] - 0s 14ms/step - loss: 41264.1953 - val_loss: 5096.1333
Epoch 1783/10000
1/1 [==============================] - 0s 14ms/step - loss: 41137.2227 - val_loss: 3354.7456
Epoch 1784/10000
1/1 [==============================] - 0s 14ms/step - loss: 40168.3281 - val_loss: 3154.4019
Epoch 1785/10000
1/1 [==============================] - 0s 14ms/step - loss: 40319.4883 - val_loss: 5218.2637
Epoch 1786/10000
1/1 [==============================] - 0s 14ms/step - loss: 40847.9102 - val_loss: 1721.0344
Epoch 1787/10000
1/1 [=====

In [35]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 50ms/step
[2782.135](test_target) - [[2332.9292]](prediction) = 449.2058007812502


In [36]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [37]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.258939,-1.397925,1.268852,-0.94647,-1.118690,-0.551011,-0.770233,-1.233828,-0.425199,1.148205,...,1.277766,1.308949,1.412346,-1.413223,-1.238512,-1.303278,0.658911,1.028795,1.089129,-1.223704
1,0.071523,0.884312,-0.093583,-0.43679,-0.189921,-0.852453,-0.642042,0.018373,1.380677,0.140893,...,-0.114016,-0.190804,-0.643250,0.752439,0.028014,0.176141,0.754232,0.325951,0.236684,-0.002080
2,1.187416,0.513613,-1.175269,1.38326,1.308611,1.403464,1.412276,1.215455,-0.955478,-1.289098,...,-1.163750,-1.118144,-0.769096,0.660784,1.210497,1.127138,-1.413142,-1.354746,-1.325813,1.225784


train_target:


,Goiás - Consumo de Cimento (t)
0,2335.717
1,2657.527
2,2901.707


test_input:


...

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.158228,-2.357097,-1.338521,0.495067,-0.996697,-1.327004,0.145995,1.141532,0.318701,1.108405,...,0.964765,-1.482779,-2.260447,0.946592,0.252129,0.101107,0.76294,-1.953922,-1.523095,-2.072592


test_target:


,Goiás - Consumo de Cimento (t)
12,2782.135


1/1 [==============================] - 0s 34ms/step
Error: 449.2058007812502




[2901.69921875,
 3026.70751953125,
 3208.161865234375,
 2963.999267578125,
 2517.340576171875,
 2203.73828125,
 2269.166748046875,
 2282.985107421875,
 2331.167724609375,
 2332.92919921875]

In [38]:
display(targets)
display(predictions)

[3061.533,
 3181.778,
 2861.8974285714285,
 2517.2907428571425,
 2193.5458285714285,
 2270.346,
 2282.903,
 2367.716,
 2438.585,
 2782.135]

[2901.69921875,
 3026.70751953125,
 3208.161865234375,
 2963.999267578125,
 2517.340576171875,
 2203.73828125,
 2269.166748046875,
 2282.985107421875,
 2331.167724609375,
 2332.92919921875]

In [39]:
mae = mean_absolute_error(predictions, targets)
mae

215.33699101562502

In [40]:
porcentage = mae/np.mean(targets)
porcentage

0.0829567882151579